In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.neural_network import MLPRegressor 
import time

In [12]:
df = pd.read_excel(r'C:\Users\TI 2023\Downloads\Data diolah darin.xlsx')
df

,Backhoe,HP(watt),Norm_MAP,RPM,Age,Load_Factor,Engine_Tier,TEMP[C],Fuel[g/s],NOx[g/s],...,NOx[g/hr],HC[g/hr],CO[g/hr],CO2[g/hr],PM[mg/hr],Nox (g/kl),HC (g/kl),CO (g/kl),CO2 (g/kl),PM (g/kl)
0,1,65621.6,0.000000,833,12,0.21,2,22,0.430000,0.024090,...,86.724000,25.848000,13.176000,4780.656000,14.4,3730.117500,1111.757728,566.717728,2.056225e+05,0.619362
1,1,65621.6,0.012346,800,12,0.21,2,22,0.620000,0.030320,...,109.152000,25.668000,16.164000,7008.264000,21.6,4694.776362,1104.015681,695.235681,3.014350e+05,0.929047
2,1,65621.6,0.012346,826,12,0.21,2,22,0.510000,0.030730,...,110.628000,10.980000,10.584000,5722.920000,21.6,4758.261138,472.264772,455.232272,2.461506e+05,0.929047
3,1,65621.6,0.012346,831,12,0.21,2,22,0.390522,0.023693,...,85.293664,20.747853,4.825082,4382.553038,21.6,3668.596780,892.393432,207.533355,1.884996e+05,0.929047
4,1,65621.6,0.012346,834,12,0.21,2,22,0.413024,0.026107,...,93.985262,6.979907,9.880556,4673.503069,21.6,4042.434273,300.215326,424.976193,2.010137e+05,0.929047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37523,8,72332.9,0.975904,1676,36,0.21,2,41,1.590000,0.058830,...,211.788000,19.908000,28.152000,17957.592000,1800.0,445343.100000,41862.100000,59197.400000,3.776083e+07,3785.000000
37524,8,72332.9,0.975904,1683,36,0.21,2,42,1.320000,0.059500,...,214.200000,19.800000,30.780000,14905.044000,1656.0,542545.340900,50151.250000,77962.397730,3.775286e+07,4194.468182
37525,8,72332.9,0.975904,1846,36,0.21,2,42,1.530000,0.059640,...,214.704000,21.744000,33.840000,17349.048000,1872.0,469179.694100,47515.850980,73948.509800,3.791183e+07,4090.768627
37526,8,72332.9,0.987952,1876,36,0.21,2,42,1.760000,0.056200,...,202.320000,22.356000,34.740000,19948.572000,1980.0,384340.943200,42468.990340,65994.485800,3.789568e+07,3761.343750


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37528 entries, 0 to 37527
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Backhoe      37528 non-null  int64  
 1   HP(watt)     37528 non-null  float64
 2   Norm_MAP     37528 non-null  float64
 3   RPM          37528 non-null  object 
 4   Age          37528 non-null  int64  
 5   Load_Factor  37528 non-null  float64
 6   Engine_Tier  37528 non-null  int64  
 7   TEMP[C]      37528 non-null  object 
 8   Fuel[g/s]    37528 non-null  float64
 9   NOx[g/s]     37528 non-null  float64
 10  HC[g/s]      37528 non-null  float64
 11  CO[g/s]      37528 non-null  float64
 12  CO2[g/s]     37528 non-null  float64
 13  PM[mg/s]     37528 non-null  float64
 14  Nox (g/l)    37528 non-null  float64
 15  HC (g/l)     37528 non-null  float64
 16  CO (g/l)     37528 non-null  float64
 17  CO2 (g/l)    37528 non-null  float64
 18  PM (g/l)     37528 non-null  float64
 19  Fuel

In [14]:
df.replace('#DIV/0!',np.nan,inplace=True)
df.dropna(inplace=True)

C:\Users\TI 2023\AppData\Local\Temp\ipykernel_14648\1019318869.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('#DIV/0!',np.nan,inplace=True)


In [15]:
X = df[['Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier']]
y = df['CO[g/s]']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

C:\Users\TI 2023\AppData\Local\Temp\ipykernel_14648\1236518287.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_train = y_train.ravel()
C:\Users\TI 2023\AppData\Local\Temp\ipykernel_14648\1236518287.py:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_test = y_test.ravel()


In [17]:
mlp = MLPRegressor()
mlp

MLPRegressor()

In [18]:
start_time = time.time()
mlp.fit(X_train, y_train)
print("Execution time: " + str((time.time() - start_time)) + ' s')

Execution time: 1.2470030784606934 s


In [19]:
y_pred = mlp.predict(X_test)

In [20]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")

RMSE: 0.34


In [21]:
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE: {mape:.2f}%")

MAPE: 2227076235106.55%


Grid

In [22]:
param = {
    'hidden_layer_sizes': range(1, 100),
    'activation':['relu','tanh', 'logistic', 'identity'],
    'alpha': np.arange(0.01, 1.0, 0.01),
    'learning_rate':['invscaling','adaptive'],
}

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)


Random

In [23]:
rs = RandomizedSearchCV(mlp, 
                            param, 
                            cv=3, 
                            scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
                            refit='RMSE',
                            n_iter=100,
                            verbose=1,
                            n_jobs=-1
                            )


start_time = time.time()
rs.fit(X_train, y_train)


print("Best: %f using %s" % (rs.best_score_, rs.best_params_))
print("Best RMSE:", rs.best_score_)
print("Best MAPE:", rs.cv_results_['mean_test_MAPE'][rs.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best: 68.626499 using {'learning_rate': 'invscaling', 'hidden_layer_sizes': 66, 'alpha': 0.64, 'activation': 'identity'}
Best RMSE: 68.62649857157501
Best MAPE: -648621532064028.6
Execution time: 26.025234937667847 s


In [33]:
best_rs = rs.best_estimator_
y_pred_rs = best_rs.predict(X_test)
rmse_rs = np.sqrt(mean_squared_error(y_test, y_pred_rs))
mape_rs = mean_absolute_percentage_error(y_test, y_pred_rs)
print(f"RMSE: {rmse_rs}")
print(f"MAPE: {mape_rs:.2f}%")

RMSE: 47.09598160756587
MAPE: 304359072409839.00%


In [34]:
best_rs

MLPRegressor(activation='identity', alpha=0.64, hidden_layer_sizes=66,
             learning_rate='invscaling')

In [25]:
pd.DataFrame(rs.cv_results_).to_excel('MLP_RS_unscal_CO.xlsx', index=False)

Bayessian

In [26]:
def mlp_bs(hidden_layer_sizes, activation, alpha, learning_rate):
    activations = ['relu','tanh', 'logistic', 'identity']
    learning_rates = ['invscaling','adaptive']
    mlpbs= MLPRegressor(
        hidden_layer_sizes=int(hidden_layer_sizes),
        activation=activations[int(activation)],
        alpha=float(alpha),
        learning_rate=learning_rates[int(learning_rate)],
    )
    scores = cross_val_score(mlpbs, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return -rmse 

In [27]:
bayes_opt = BayesianOptimization(
    mlp_bs,
    {
        'hidden_layer_sizes': (1, 100),
        'activation': (0, 3),
        'alpha': (0.01, 1.0),
        'learning_rate': (0, 1)
    }
)

In [28]:
start_time = time.time()
bayes_opt.maximize(init_points=10, n_iter=100)
print("Execution time: " + str((time.time() - start_time)) + ' s')

|   iter    |  target   | activa... |   alpha   | hidden... | learni... |
-------------------------------------------------------------------------
| 1         | -0.02748  | 2.426     | 0.2685    | 90.79     | 0.08536   |
| 2         | -0.02247  | 2.014     | 0.4183    | 23.45     | 0.2194    |
| 3         | -0.02242  | 1.162     | 0.6648    | 18.7      | 0.1525    |
| 4         | -0.0226   | 2.233     | 0.797     | 43.13     | 0.5478    |
| 5         | -0.02502  | 1.323     | 0.805     | 4.65      | 0.0201    |
| 6         | -0.02304  | 2.62      | 0.07026   | 98.78     | 0.7373    |
| 7         | -0.02414  | 1.679     | 0.2962    | 30.31     | 0.5341    |
| 8         | -1.098    | 0.1305    | 0.02994   | 36.56     | 0.6084    |
| 9         | -6.738    | 0.7689    | 0.3636    | 82.04     | 0.9207    |
| 10        | -0.02285  | 1.814     | 0.6389    | 32.66     | 0.7398    |
| 11        | -38.88    | 3.0       | 1.0       | 53.97     | 0.0       |
| 12        | -15.33    | 3.0       | 

c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


| 29        | -2.981    | 0.0       | 1.0       | 3.222     | 0.0       |
| 30        | -2.723    | 0.4849    | 0.8079    | 97.77     | 0.9498    |
| 31        | -0.02242  | 1.084     | 0.6867    | 21.16     | 0.336     |
| 32        | -2.134    | 3.0       | 0.01      | 20.18     | 1.0       |
| 33        | -0.02247  | 2.934     | 0.01      | 45.64     | 0.0       |
| 34        | -31.34    | 3.0       | 0.01      | 27.13     | 0.0       |
| 35        | -1.697    | 0.0       | 0.01      | 31.64     | 1.0       |
| 36        | -4.402    | 0.0       | 1.0       | 34.24     | 0.0       |
| 37        | -0.2771   | 0.0       | 1.0       | 62.73     | 1.0       |
| 38        | -1.851    | 0.0       | 0.01      | 42.16     | 1.0       |
| 39        | -24.45    | 3.0       | 1.0       | 16.62     | 1.0       |
| 40        | -0.02423  | 2.176     | 1.0       | 47.92     | 1.0       |
| 41        | -37.28    | 3.0       | 0.01      | 35.18     | 1.0       |
| 42        | -0.8746   | 0.0       | 

c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


| 88        | -0.6044   | 0.0       | 0.01      | 4.598     | 1.0       |
| 89        | -0.9796   | 0.0       | 1.0       | 6.069     | 0.0       |


c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


| 90        | -0.3616   | 0.8143    | 0.01      | 1.0       | 1.0       |
| 91        | -29.16    | 3.0       | 0.01      | 60.73     | 0.0       |
| 92        | -0.02337  | 1.895     | 0.01      | 57.94     | 1.0       |
| 93        | -0.02255  | 1.804     | 1.0       | 59.13     | 1.0       |
| 94        | -3.105    | 0.0       | 0.01      | 62.17     | 0.0       |
| 95        | -0.02242  | 1.156     | 0.01      | 8.951     | 1.0       |
| 96        | -0.02243  | 1.605     | 1.0       | 72.29     | 0.0       |
| 97        | -0.02244  | 2.011     | 0.01      | 46.61     | 1.0       |
| 98        | -1.629    | 0.0       | 0.01      | 96.24     | 0.0       |
| 99        | -13.0     | 0.0       | 0.01      | 51.33     | 1.0       |
| 100       | -0.02242  | 2.192     | 1.0       | 1.0       | 0.0       |
| 101       | -1.528    | 0.0       | 1.0       | 40.28     | 0.0       |
| 102       | -8.253    | 0.3463    | 1.0       | 31.9      | 0.0       |
| 103       | -2.104    | 0.0       | 

In [29]:
best_iter = np.argmin([res['target'] for res in bayes_opt.res])

In [30]:
best_params = bayes_opt.res[best_iter]['params']

print("Best Parameters:", best_params)

Best Parameters: {'activation': 3.0, 'alpha': 0.01, 'hidden_layer_sizes': 87.32383234110733, 'learning_rate': 0.0}


In [31]:
activations = ['relu','tanh', 'logistic', 'identity']
learning_rates = ['invscaling','adaptive']

best_bs = MLPRegressor(
    hidden_layer_sizes=int(best_params['hidden_layer_sizes']),
    activation=activations[int(best_params['activation'])],  # Convert int to string
    alpha=float(best_params['alpha']),
    learning_rate=learning_rates[int(best_params['learning_rate'])]  # Convert int to string
)
best_bs.fit(X_train, y_train)

MLPRegressor(activation='identity', alpha=0.01, hidden_layer_sizes=87,
             learning_rate='invscaling')

In [32]:
y_pred_bs = best_bs.predict(X_test)
rmse_bs = np.sqrt(mean_squared_error(y_test, y_pred_bs))
mape_bs = mean_absolute_percentage_error(y_test, y_pred_bs)

print(f"RMSE: {rmse_bs}")
print(f"MAPE: {mape_bs:.2f}%")

RMSE: 2.5594389139992306
MAPE: 17500526527468.18%
